下载和解压数据集，只运行一次就可以了

In [ ]:
'''!mkdir DataSet
!mkdir saved_models
!wget -c "http://oss.hacking-linux.com/Dogs_vs_Cats/train.zip" -O DataSet/train.zip
!wget -c "http://oss.hacking-linux.com/Dogs_vs_Cats/test.zip" -O DataSet/test.zip
!wget -c "http://oss.hacking-linux.com/Dogs_vs_Cats/sample_submission.csv" -O DataSet/sample_submission.csv
!unzip DataSet/train.zip -d DataSet/
!unzip DataSet/test.zip -d DataSet/
'''

导入需要的库

In [23]:
import os
import re
import cv2
import csv
import h5py
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
from random import choice
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.utils import shuffle
from keras.preprocessing import image
from keras.applications import xception
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Lambda
from keras.layers import Input, Dropout, Flatten, Dense, BatchNormalization
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.models import *
%matplotlib inline

In [24]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

In [25]:
height, width, nb_class =224,224,2
batch_size=16

In [26]:
gen = ImageDataGenerator()

读入图片，依据图片的命名规则**cat.N.jpg**记录标签

In [27]:
from keras.preprocessing import image
train_filenames = os.listdir('DataSet/train')
test_filenames = os.listdir('DataSet/test')
def get_data():
    x_train=[]
    for i in train_filenames:
        train_label = np.array([0 if i[:3]=='cat' else  1 for i in train_filenames])
        x=image.load_img('DataSet/train/'+i,target_size=(224,224))
        x=image.img_to_array(x)
        x_train.append(x)
    x_train=np.array(x_train)
    x_test=[]
    for i in test_filenames:
        x=image.load_img('DataSet/test/'+i,target_size=(224,224))
        x=image.img_to_array(x)
        x_test.append(x)
    x_test=np.array(x_test)
    print('finish')
    return x_train,train_label,x_test
    

In [28]:
x_train, train_label, x_test=get_data()

finish


导入我们的预训练模型并查看模型形状

In [30]:
from keras.applications.resnet50 import ResNet50

input_tensor = Input((height, width, 3))
x = input_tensor
base_model = ResNet50(input_tensor = input_tensor, weights = 'imagenet', include_top = False)

resnet50_model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
resnet50_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
x_train_resnet = resnet50_model.predict_generator(gen.flow(x_train, train_label,shuffle = False, batch_size = batch_size), verbose = 1)

 719/1563 [============>.................] - ETA: 2:07

In [ ]:
x_test_resnet = resnet50_model.predict_generator(gen.flow(x_test,shuffle = False, batch_size = batch_size), verbose = 1)

使用与训练模型进行预测并到处权重存储，这样我们每次预测只需要读入这些数据就可以了

In [ ]:
with h5py.File('saved_models/weights.resnet.hdf5') as fp:
        fp.create_dataset('x_train', data = x_train_resnet)
        fp.create_dataset('x_test', data = x_test_resnet)
        fp.create_dataset('train_label',data = train_label)

In [34]:
X_train = []
X_test = []
train_label=[]
with h5py.File('saved_models/weights.resnet.hdf5', 'r') as fp:
    X_train.append(np.array(fp['x_train']))
    X_test.append(np.array(fp['x_test']))
    train_label=np.array(fp['train_label'])    

In [35]:
X_train = np.concatenate(X_train, axis = 1)
X_test = np.concatenate(X_test, axis = 1)
X_train, train_label = shuffle(X_train, train_label)

编译基本模型，按照文档给的序贯模型就可以了

In [36]:
input_tensor = Input(X_train.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

"\ninput_tensor = Input(X_train.shape[1:])\nmodel = Model(input_tensor, Dropout(0.5)(input_tensor))\nmodel = Model(model.input, Dense(2, activation = 'sigmoid')(model.output))\nmodel.summary()\n\n\nx1 = resnet50_model.output\nx = Dense(1024, activation='relu')(x)\npredictions = Dense(2, activation='softmax')(x)\nmodel = Model(inputs=resnet50_model.input, outputs=predictions)\n"

In [37]:
model.compile(loss='binary_crossentropy', optimizer='adadelta', metrics=['accuracy'])

In [38]:
batch_size = 64
model.fit(X_train, train_label, validation_split = 0.3,epochs = 30, batch_size = batch_size, verbose = 1)


Train on 17500 samples, validate on 7500 samples
Epoch 1/30
17500/17500 [==============================] - 5s 284us/step - loss: 0.1635 - acc: 0.9322 - val_loss: 0.0674 - val_acc: 0.9763
Epoch 2/30
17500/17500 [==============================] - 1s 75us/step - loss: 0.0852 - acc: 0.9673 - val_loss: 0.0703 - val_acc: 0.9717
Epoch 3/30
17500/17500 [==============================] - 1s 75us/step - loss: 0.0797 - acc: 0.9694 - val_loss: 0.0557 - val_acc: 0.9776
Epoch 4/30
17500/17500 [==============================] - 1s 75us/step - loss: 0.0776 - acc: 0.9705 - val_loss: 0.0554 - val_acc: 0.9784
Epoch 5/30
17500/17500 [==============================] - 1s 76us/step - loss: 0.0726 - acc: 0.9725 - val_loss: 0.0554 - val_acc: 0.9781
Epoch 6/30
17500/17500 [==============================] - 1s 75us/step - loss: 0.0715 - acc: 0.9731 - val_loss: 0.0547 - val_acc: 0.9784
Epoch 7/30
17500/17500 [==============================] - 1s 74us/step - loss: 0.0698 - acc: 0.9723 - val_loss: 0.0549 - val_acc

'\nmodel.fit_generator(gen.flow(X_train, train_label, batch_size=batch_size),\n                    epochs=10, verbose=1, validation_data=(X_test, test_label))'

97.7%左右的正确率，虽然还不错，但是和理想中还有一定的差距，接下来进行细节调整